# Data Processing

## Data Summary
- Eurovision votes - votes.csv
    - Eurovisionworld fan website
    - ISO alpha-2 country codes
- Country codes
    - TODO: Find dataset
    - Can webscrape from official [ISO website](https://www.iso.org/obp/ui/#search)
    - Made country_codes_raw.csv from copy-pasting
    - Need ISO alpha-2 and ISO alpha-3
- Country capitals
    - TODO: Find dataset
- Religious demographic data
    - WRP_national.csv
    - ISO alpha-3 country codes
- GDP data
    - World_Bank_Data.csv
    - ISO alpha-3 country codes
    - TODO: NaNs encoded as '..'

## TODO:
- Categorical relgious demographic variables - most popular
- Difference/average in GDP per capita between countries?
- Add flexible feature selection

# Utilities
- TODO: Move to .py files?

In [31]:
import pandas as pd

In [32]:
votes = pd.read_csv("../data/votes.csv")

exclude_self_votes = True
if exclude_self_votes:
    votes = votes[votes["from_country"] != votes["to_country"]] #exclude self-votes

## Clean Country Codes Data

In [33]:
codes_raw = pd.read_csv("../data/country_codes_raw.csv")
codes_raw.head()

,name_fr,iso-alpha-2,iso-alpha-3,numeric
0,Afghanistan (l'),AF,AFG,4.0
1,NaN,NaN,NaN,NaN
2,Albanie (l'),AL,ALB,8.0
3,NaN,NaN,NaN,NaN
4,Algérie (l'),DZ,DZA,12.0


In [34]:
def get_clean_codes(codes_raw):
    codes = codes_raw.dropna()
    codes = codes[["iso-alpha-2", "iso-alpha-3"]]
    codes["iso-alpha-2"] = codes["iso-alpha-2"].str.lower()
    return codes

In [35]:
country_codes = get_clean_codes(codes_raw)
country_codes.head()

,iso-alpha-2,iso-alpha-3
0,af,AFG
2,al,ALB
4,dz,DZA
6,as,ASM
8,ad,AND


In [36]:
countries = pd.read_csv("../data/country_info.csv")
countries.head()

,name,code,capital
0,Albania,al,Tirana
1,Armenia,am,Yerevan
2,Australia,au,Canberra
3,Austria,at,Vienna
4,Azerbaijan,az,Baku


In [37]:
merged_countries = countries.merge(country_codes, left_on="code", right_on="iso-alpha-2")
merged_countries = merged_countries.drop("code", axis=1)
merged_countries.head()

,name,capital,iso-alpha-2,iso-alpha-3
0,Albania,Tirana,al,ALB
1,Armenia,Yerevan,am,ARM
2,Australia,Canberra,au,AUS
3,Austria,Vienna,at,AUT
4,Azerbaijan,Baku,az,AZE


In [38]:
cow_codes = pd.read_csv("../data/COW-country-codes.csv")
merged_countries = merged_countries.merge(cow_codes.drop_duplicates(), left_on="name", right_on="StateNme", how="left")
merged_countries = merged_countries.drop(['CCode', 'StateNme'], axis=1)
merged_countries.tail()


,name,capital,iso-alpha-2,iso-alpha-3,StateAbb
45,North Macedonia,Skopje,mk,MKD,MAC
46,Montenegro,Podgorica,me,MNE,MNG
47,Russia,Moscow,ru,RUS,RUS
48,Slovakia,Bratislava,sk,SVK,SLO
49,Turkiye,Ankara,tr,TUR,TUR


## Country Distances

In [39]:
import geopandas as gpd
from shapely.geometry import Point

# Load the dataset
from geopandas.datasets import get_path

cities = gpd.read_file(get_path('naturalearth_cities'))

/tmp/ipykernel_17358/3332143984.py:7: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_cities' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  cities = gpd.read_file(get_path('naturalearth_cities'))


In [40]:
# Filter for European capitals (manually or with an additional dataset)
european_capitals = countries["capital"]

european_cities = cities[cities['name'].isin(european_capitals)]

# Reproject to a metric CRS (EPSG:3035)
european_cities = european_cities.to_crs(epsg=3035)

# Compute pairwise distances
distances = european_cities.geometry.apply(
    lambda city: european_cities.distance(city)
)
distance_df = pd.DataFrame(distances.values.tolist(),
                           index=european_cities['name'],
                           columns=european_cities['name'])
distance_df_km = distance_df / 1000

In [41]:
european_cities['lat'] = european_cities["geometry"].x
european_cities['lon'] = european_cities["geometry"].y
european_cities = european_cities.rename(columns={"name":"city"})

In [42]:
merged_countries = merged_countries.merge(european_cities, left_on="capital", right_on="city")
merged_countries

,name,capital,iso-alpha-2,iso-alpha-3,StateAbb,city,geometry,lat,lon
0,Albania,Tirana,al,ALB,ALB,Tirana,POINT (5143798.283 2078711.221),5.143798e+06,2.078711e+06
1,Armenia,Yerevan,am,ARM,ARM,Yerevan,POINT (7160091.412 2563395.278),7.160091e+06,2.563395e+06
2,Australia,Canberra,au,AUS,AUL,Canberra,POINT (16181012.366 6106953.388),1.618101e+07,6.106953e+06
3,Austria,Vienna,at,AUT,AUS,Vienna,POINT (4793664.530 2807989.732),4.793665e+06,2.807990e+06
4,Azerbaijan,Baku,az,AZE,AZE,Baku,POINT (7544318.916 2803833.748),7.544319e+06,2.803834e+06
5,Belgium,Brussels,be,BEL,BEL,Brussels,POINT (3922116.594 3095876.588),3.922117e+06,3.095877e+06
6,Croatia,Zagreb,hr,HRV,CRO,Zagreb,POINT (4787502.800 2539616.642),4.787503e+06,2.539617e+06
7,Cyprus,Nicosia,cy,CYP,CYP,Nicosia,POINT (6435608.312 1669793.415),6.435608e+06,1.669793e+06
8,Czechia,Prague,cz,CZE,CZR,Prague,POINT (4637345.485 3006721.767),4.637345e+06,3.006722e+06
9,Denmark,København,dk,DNK,DEN,København,POINT (4482192.308 3622431.430),4.482192e+06,3.622431e+06


## GDP Statistics

In [43]:
gdp_raw = pd.read_csv("../data/World_Bank_Data.csv")
gdp_raw

,Series Name,Series Code,Country Name,Country Code,1960 [YR1960],1961 [YR1961],1962 [YR1962],1963 [YR1963],1964 [YR1964],1965 [YR1965],...,2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022],2023 [YR2023]
0,"Population, total",SP.POP.TOTL,Afghanistan,AFG,9035043,9214083,9404406,9604487,9814318,10036008,...,32792523,33831764,34700612,35688935,36743039,37856121,39068979,40000412,40578842,41454761
1,"Population, total",SP.POP.TOTL,Algeria,DZA,11424922,11628883,11800771,11982118,12179813,12365976,...,39205031,40019529,40850721,41689299,42505035,43294546,44042091,44761099,45477389,46164219
2,"Population, total",SP.POP.TOTL,Angola,AGO,5231654,5301583,5354310,5408320,5464187,5521981,...,27160769,28157798,29183070,30234839,31297155,32375632,33451132,34532429,35635029,36749906
3,"Population, total",SP.POP.TOTL,Antigua and Barbuda,ATG,55603,56540,57336,58138,59020,59970,...,88765,89409,89969,90468,90926,91364,91846,92349,92840,93316
4,"Population, total",SP.POP.TOTL,Argentina,ARG,20386045,20726276,21072538,21421705,21769453,22112629,...,43024071,43477012,43900313,44288894,44654882,44973465,45191965,45312281,45407904,45538401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9051,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9053,Data from database: World Development Indicators,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
gdp_data = gdp_raw.drop(["Country Name", "Series Code"], axis=1)
gdp_data = gdp_data.iloc[:-5]
# gdp_data.unstack()
gdp_data = gdp_data.set_index(["Country Code","Series Name"])
gdp_data = gdp_data.stack().unstack(1)
# gdp_data["Series Name"].unique()
# gdp_raw["Series Name"].unique()

# PICK GDP DATA COLUMNS HERE
gdp_data = gdp_data[['GDP per capita (current US$)', 'Population, total']].reset_index()
gdp_data["Year"] = gdp_data["level_1"].str.split().str.get(0)
gdp_data["Year"] = pd.to_numeric(gdp_data["Year"])
gdp_data = gdp_data.drop("level_1", axis=1)
gdp_data
# gdp_data[gdp_data["Country Code"] == "DEU"]

Series Name,Country Code,GDP per capita (current US$),"Population, total",Year
0,ABW,..,54922,1960
1,ABW,..,55578,1961
2,ABW,..,56320,1962
3,ABW,..,57002,1963
4,ABW,..,57619,1964
...,...,...,...,...
11579,ZWE,1684.02790388182,15271368,2019
11580,ZWE,1730.41348946953,15526888,2020
11581,ZWE,1724.38773104902,15797210,2021
11582,ZWE,2040.55245910726,16069056,2022


In [45]:
europe_gdp_data = merged_countries.merge(gdp_data, left_on="iso-alpha-3", right_on="Country Code")

In [46]:
europe_gdp_data

,name,capital,iso-alpha-2,iso-alpha-3,StateAbb,city,geometry,lat,lon,Country Code,GDP per capita (current US$),"Population, total",Year
0,Albania,Tirana,al,ALB,ALB,Tirana,POINT (5143798.283 2078711.221),5.143798e+06,2.078711e+06,ALB,..,1608800,1960
1,Albania,Tirana,al,ALB,ALB,Tirana,POINT (5143798.283 2078711.221),5.143798e+06,2.078711e+06,ALB,..,1659800,1961
2,Albania,Tirana,al,ALB,ALB,Tirana,POINT (5143798.283 2078711.221),5.143798e+06,2.078711e+06,ALB,..,1711319,1962
3,Albania,Tirana,al,ALB,ALB,Tirana,POINT (5143798.283 2078711.221),5.143798e+06,2.078711e+06,ALB,..,1762621,1963
4,Albania,Tirana,al,ALB,ALB,Tirana,POINT (5143798.283 2078711.221),5.143798e+06,2.078711e+06,ALB,..,1814135,1964
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3131,Turkiye,Ankara,tr,TUR,TUR,Ankara,POINT (6252317.487 2164797.267),6.252317e+06,2.164797e+06,TUR,9215.44049888114,82579440,2019
3132,Turkiye,Ankara,tr,TUR,TUR,Ankara,POINT (6252317.487 2164797.267),6.252317e+06,2.164797e+06,TUR,8638.73903848102,83384680,2020
3133,Turkiye,Ankara,tr,TUR,TUR,Ankara,POINT (6252317.487 2164797.267),6.252317e+06,2.164797e+06,TUR,9743.21277804855,84147318,2021
3134,Turkiye,Ankara,tr,TUR,TUR,Ankara,POINT (6252317.487 2164797.267),6.252317e+06,2.164797e+06,TUR,10674.504157865,84979913,2022


In [47]:
# votes.merge(europe_gdp_data, left_on=["from_country_id", "year"], right_on=["iso-alpha-2", "Year"])

## Religious Demographic Data

In [48]:
wrp_raw = pd.read_csv("../data/WRP_national.csv")
wrp_raw.head()

,year,state,name,chrstprot,chrstcat,chrstorth,chrstang,chrstothr,chrstgen,judorth,...,othrgenpct,sumreligpct,total,dualrelig,datatype,sourcereliab,recreliab,reliabilevel,Version,sourcecode
0,1945,2,USA,66069671,38716742,1121898,2400000,1956807,110265118,821489,...,0.0039,0.9961,1.0000,0,34,2,10,Medium,1.1,13
1,1950,2,USA,73090083,42635882,3045420,3045420,1177214,122994019,1078078,...,0.0041,0.9959,1.0000,0,34,6,28,Low,1.1,18
2,1955,2,USA,79294628,46402368,3454916,2572767,2277091,134001770,944000,...,0.0193,0.9807,0.9999,0,134,5,10,Medium,1.1,15
3,1960,2,USA,90692928,50587880,3334535,2710065,2908939,150234347,973500,...,0.0076,0.9924,0.9999,0,134,2,10,Medium,1.1,13
4,1965,2,USA,94165803,64761783,4792868,2822149,973155,167515758,991200,...,0.0030,0.9970,1.0001,0,134,8,28,Low,1.1,20


In [49]:
europe_religion_data = merged_countries.merge(wrp_raw, left_on='StateAbb', right_on="name")

# PICK RELIGION COLUMNS HERE

religion_cols = ['chrstprotpct', 'chrstcatpct', 'chrstorthpct', 'judgenpct', 'islmgenpct', 'nonreligpct']
test_data = europe_religion_data[["year", "iso-alpha-3"]+religion_cols]
# test_data.merge(europe_gdp_data[["year", "iso-alpha-3"]], left_on=["year", "iso-alpha-3"], right_on=["Year", "iso-alpha-3"], how="right")
europe_religion_data.columns

Index(['name_x', 'capital', 'iso-alpha-2', 'iso-alpha-3', 'StateAbb', 'city',
       'geometry', 'lat', 'lon', 'year', 'state', 'name_y', 'chrstprot',
       'chrstcat', 'chrstorth', 'chrstang', 'chrstothr', 'chrstgen', 'judorth',
       'jdcons', 'judref', 'judothr', 'judgen', 'islmsun', 'islmshi',
       'islmibd', 'islmnat', 'islmalw', 'islmahm', 'islmothr', 'islmgen',
       'budmah', 'budthr', 'budothr', 'budgen', 'zorogen', 'hindgen',
       'sikhgen', 'shntgen', 'bahgen', 'taogen', 'jaingen', 'confgen',
       'syncgen', 'anmgen', 'nonrelig', 'othrgen', 'sumrelig', 'pop',
       'chrstprotpct', 'chrstcatpct', 'chrstorthpct', 'chrstangpct',
       'chrstothrpct', 'chrstgenpct', 'judorthpct', 'judconspct', 'judrefpct',
       'judothrpct', 'judgenpct', 'islmsunpct', 'islmshipct', 'islmibdpct',
       'islmnatpct', 'islmalwpct', 'islmahmpct', 'islmothrpct', 'islmgenpct',
       'budmahpct', 'budthrpct', 'budothrpct', 'budgenpct', 'zorogenpct',
       'hindgenpct', 'sikhgenpct', 'sh

In [50]:
test_merged = test_data.merge(europe_gdp_data, how="right", left_on=["year", "iso-alpha-3"], right_on=["Year" ,"iso-alpha-3"])
test_merged = test_merged.drop("year", axis=1)
test_merged = test_merged.set_index(["Year", "iso-alpha-3"])


# Linear interpolation between years :)
# test_merged[["chrstprot", "chrstcat"]] = test_merged[["chrstprot", "chrstcat"]].interpolate()
# test_merged["chrstprot"] = test_merged["chrstprot"].interpolate()
test_merged.head()

,,chrstprotpct,chrstcatpct,chrstorthpct,judgenpct,islmgenpct,nonreligpct,name,capital,iso-alpha-2,StateAbb,city,geometry,lat,lon,Country Code,GDP per capita (current US$),"Population, total"
Year,iso-alpha-3,,,,,,,,,,,,,,,,,
1960,ALB,0.0,0.074,0.2048,0.0002,0.5689,0.15,Albania,Tirana,al,ALB,Tirana,POINT (5143798.283 2078711.221),5.143798e+06,2.078711e+06,ALB,..,1608800
1961,ALB,NaN,NaN,NaN,NaN,NaN,NaN,Albania,Tirana,al,ALB,Tirana,POINT (5143798.283 2078711.221),5.143798e+06,2.078711e+06,ALB,..,1659800
1962,ALB,NaN,NaN,NaN,NaN,NaN,NaN,Albania,Tirana,al,ALB,Tirana,POINT (5143798.283 2078711.221),5.143798e+06,2.078711e+06,ALB,..,1711319
1963,ALB,NaN,NaN,NaN,NaN,NaN,NaN,Albania,Tirana,al,ALB,Tirana,POINT (5143798.283 2078711.221),5.143798e+06,2.078711e+06,ALB,..,1762621
1964,ALB,NaN,NaN,NaN,NaN,NaN,NaN,Albania,Tirana,al,ALB,Tirana,POINT (5143798.283 2078711.221),5.143798e+06,2.078711e+06,ALB,..,1814135


In [51]:
test_interp = test_merged[religion_cols]
test_interp = test_interp.unstack()
test_interp = test_interp.interpolate()
test_interp = test_interp.stack(future_stack=True)
test_merged = test_interp.merge(europe_gdp_data, how="right", left_on=["Year", "iso-alpha-3"], right_on=["Year" ,"iso-alpha-3"])

In [52]:
merged_1 = votes.merge(test_merged, left_on=["year", "from_country_id"], right_on=["Year", "iso-alpha-2"])
merged_2 = merged_1.merge(test_merged, left_on=["year", "to_country_id"], right_on=["Year", "iso-alpha-2"], suffixes=("_from", "_to"))
# set(merged_1["to_country_id"].unique()) - set(merged_1["from_country_id"].unique())
merged_2

,year,round,from_country_id,to_country_id,from_country,to_country,total_points,tele_points,jury_points,Year_from,...,capital_to,iso-alpha-2_to,StateAbb_to,city_to,geometry_to,lat_to,lon_to,Country Code_to,GDP per capita (current US$)_to,"Population, total_to"
0,1960,final,at,fr,at,fr,1,NaN,NaN,1960,...,Paris,fr,FRN,Paris,POINT (3760846.526 2889643.951),3.760847e+06,2.889644e+06,FRA,1302.53149729892,47412964
1,1960,final,at,gb,at,gb,3,NaN,NaN,1960,...,London,gb,UKG,London,POINT (3620981.181 3203213.160),3.620981e+06,3.203213e+06,GBR,1397.5948032844,52400000
2,1960,final,at,mc,at,mc,0,NaN,NaN,1960,...,Monaco,mc,MNC,Monaco,POINT (4111647.583 2295892.475),4.111648e+06,2.295892e+06,MCO,..,21808
3,1960,final,at,no,at,no,1,NaN,NaN,1960,...,Oslo,no,NOR,Oslo,POINT (4362948.241 4091117.366),4.362948e+06,4.091117e+06,NOR,1451.2850779894,3581239
4,1960,final,at,de,at,de,2,NaN,NaN,1960,...,Berlin,de,GMY,Berlin,POINT (4551650.878 3273669.652),4.551651e+06,3.273670e+06,DEU,1162.12266831735,72814900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47416,2023,final,gb,si,gb,si,1,0.0,1.0,2023,...,Ljubljana,si,SLV,Ljubljana,POINT (4670581.857 2559757.799),4.670582e+06,2.559758e+06,SVN,32610.110922729,2120461
47417,2023,final,gb,al,gb,al,0,0.0,0.0,2023,...,Tirana,al,ALB,Tirana,POINT (5143798.283 2078711.221),5.143798e+06,2.078711e+06,ALB,8575.17113446213,2745972
47418,2023,final,gb,pt,gb,pt,0,0.0,0.0,2023,...,Lisbon,pt,POR,Lisbon,POINT (2664943.983 1947183.552),2.664944e+06,1.947184e+06,PRT,27331.2094945255,10578174
47419,2023,final,gb,rs,gb,rs,0,0.0,0.0,2023,...,Belgrade,rs,NaN,Belgrade,POINT (5147210.901 2470142.208),5.147211e+06,2.470142e+06,SRB,12281.5058488303,6623183


## Country Distances

In [53]:
# Filter for European capitals (manually or with an additional dataset)
european_capitals = countries["capital"]

european_cities = cities[cities['name'].isin(european_capitals)]

# Reproject to a metric CRS (EPSG:3035)
european_cities = european_cities.to_crs(epsg=3035)

# Compute pairwise distances
distances = european_cities.geometry.apply(
    lambda city: european_cities.distance(city)
)
distance_df = pd.DataFrame(distances.values.tolist(),
                           index=european_cities['name'],
                           columns=european_cities['name'])
distance_df_km = distance_df / 1000

In [54]:
distance_stack = distance_df_km.stack().rename("distance").to_frame()
distance_stack.index = distance_stack.index.rename(["from", "to"])
distance_stack = distance_stack.reset_index()
distance_stack

,from,to,distance
0,San Marino,San Marino,0.000000
1,San Marino,Luxembourg,793.187699
2,San Marino,Monaco,406.385851
3,San Marino,Ljubljana,286.797948
4,San Marino,Bratislava,592.189117
...,...,...,...
2396,Paris,Vienna,1036.040752
2397,Paris,London,343.348168
2398,Paris,Moscow,2486.443932
2399,Paris,Rome,1106.651465


In [55]:
vote_dists = merged_2.merge(distance_stack, left_on=["capital_from", "capital_to"], right_on=["from", "to"], how="left")
vote_dists = vote_dists.drop(["from", "to"], axis=1)
vote_dists

,year,round,from_country_id,to_country_id,from_country,to_country,total_points,tele_points,jury_points,Year_from,...,iso-alpha-2_to,StateAbb_to,city_to,geometry_to,lat_to,lon_to,Country Code_to,GDP per capita (current US$)_to,"Population, total_to",distance
0,1960,final,at,fr,at,fr,1,NaN,NaN,1960,...,fr,FRN,Paris,POINT (3760846.526 2889643.951),3.760847e+06,2.889644e+06,FRA,1302.53149729892,47412964,1036.040752
1,1960,final,at,gb,at,gb,3,NaN,NaN,1960,...,gb,UKG,London,POINT (3620981.181 3203213.160),3.620981e+06,3.203213e+06,GBR,1397.5948032844,52400000,1237.492544
2,1960,final,at,mc,at,mc,0,NaN,NaN,1960,...,mc,MNC,Monaco,POINT (4111647.583 2295892.475),4.111648e+06,2.295892e+06,MCO,..,21808,852.872041
3,1960,final,at,no,at,no,1,NaN,NaN,1960,...,no,NOR,Oslo,POINT (4362948.241 4091117.366),4.362948e+06,4.091117e+06,NOR,1451.2850779894,3581239,1353.489212
4,1960,final,at,de,at,de,2,NaN,NaN,1960,...,de,GMY,Berlin,POINT (4551650.878 3273669.652),4.551651e+06,3.273670e+06,DEU,1162.12266831735,72814900,524.812724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47416,2023,final,gb,si,gb,si,1,0.0,1.0,2023,...,si,SLV,Ljubljana,POINT (4670581.857 2559757.799),4.670582e+06,2.559758e+06,SVN,32610.110922729,2120461,1231.136216
47417,2023,final,gb,al,gb,al,0,0.0,0.0,2023,...,al,ALB,Tirana,POINT (5143798.283 2078711.221),5.143798e+06,2.078711e+06,ALB,8575.17113446213,2745972,1893.007274
47418,2023,final,gb,pt,gb,pt,0,0.0,0.0,2023,...,pt,POR,Lisbon,POINT (2664943.983 1947183.552),2.664944e+06,1.947184e+06,PRT,27331.2094945255,10578174,1578.485825
47419,2023,final,gb,rs,gb,rs,0,0.0,0.0,2023,...,rs,NaN,Belgrade,POINT (5147210.901 2470142.208),5.147211e+06,2.470142e+06,SRB,12281.5058488303,6623183,1693.153915


# Music Tastes

In [56]:
jaccard_genre_raw = pd.read_csv("../data/jaccard_normalized_genres_similarity.csv")
jaccard_artist_raw = pd.read_csv("../data/jaccard_normalized_artists_similarity.csv")
dice_artist_raw = pd.read_csv("../data/dice_sorensen_normalized_artists_similarity.csv")
dice_genre_raw = pd.read_csv("../data/dice_sorensen_normalized_genres_similarity.csv")
jaccard_artist_raw.head()

,country,at,be,dk,fr,de,it,lu,nl,ch,...,bg,md,am,cz,ge,me,rs,az,sm,au
0,at,1.000000,0.250000,0.090909,0.090909,0.666667,0.034483,0.333333,0.250000,0.428571,...,0.090909,0.200000,0.200000,0.153846,0.200000,0.200000,0.071429,0.200000,0.200000,0.224490
1,be,0.250000,1.000000,0.111111,0.250000,0.224490,0.034483,0.578947,0.304348,0.578947,...,0.132075,0.200000,0.200000,0.153846,0.250000,0.200000,0.132075,0.200000,0.200000,0.224490
2,dk,0.090909,0.111111,1.000000,0.071429,0.090909,0.034483,0.111111,0.090909,0.111111,...,0.071429,0.111111,0.111111,0.090909,0.111111,0.111111,0.071429,0.111111,0.111111,0.111111
3,fr,0.090909,0.250000,0.071429,1.000000,0.090909,0.034483,0.176471,0.071429,0.200000,...,0.132075,0.111111,0.111111,0.090909,0.153846,0.111111,0.132075,0.111111,0.111111,0.090909
4,de,0.666667,0.224490,0.090909,0.090909,1.000000,0.034483,0.304348,0.224490,0.363636,...,0.090909,0.200000,0.200000,0.153846,0.176471,0.200000,0.071429,0.200000,0.200000,0.176471


In [57]:
jaccard_artist = jaccard_artist_raw.set_index("country").stack().rename("jaccard_genre")
jaccard_genre = jaccard_genre_raw.set_index("country").stack().rename("jaccard_artists")
dice_artist = dice_artist_raw.set_index("country").stack().rename("dice_genre")
dice_genre = dice_genre_raw.set_index("country").stack().rename("dice_artists")
music_similarity = pd.concat([jaccard_genre, jaccard_artist, dice_artist, dice_genre], axis=1)
music_similarity.index = music_similarity.index.rename(["from", "to"])
music_similarity = music_similarity.reset_index()
music_similarity.head()

# distance_stack = distance_df_km.stack().rename("distance").to_frame()
# distance_stack.index = distance_stack.index.rename(["from", "to"])
# distance_stack = distance_stack.reset_index()

,from,to,jaccard_artists,jaccard_genre,dice_genre,dice_artists
0,at,at,1.000000,1.000000,1.000000,1.000000
1,at,be,0.363636,0.250000,0.400000,0.533333
2,at,dk,0.166667,0.090909,0.166667,0.285714
3,at,fr,0.153846,0.090909,0.166667,0.266667
4,at,de,0.666667,0.666667,0.800000,0.800000


In [58]:
vote_dists2 = vote_dists.merge(music_similarity, left_on=["from_country_id", "to_country_id"], right_on=["from", "to"], how="left")
vote_dists2 = vote_dists2.drop(["from", "to"], axis=1)
vote_dists2.head()

,year,round,from_country_id,to_country_id,from_country,to_country,total_points,tele_points,jury_points,Year_from,...,lat_to,lon_to,Country Code_to,GDP per capita (current US$)_to,"Population, total_to",distance,jaccard_artists,jaccard_genre,dice_genre,dice_artists
0,1960,final,at,fr,at,fr,1,NaN,NaN,1960,...,3.760847e+06,2.889644e+06,FRA,1302.53149729892,47412964,1036.040752,0.153846,0.090909,0.166667,0.266667
1,1960,final,at,gb,at,gb,3,NaN,NaN,1960,...,3.620981e+06,3.203213e+06,GBR,1397.5948032844,52400000,1237.492544,0.302326,0.224490,0.366667,0.464286
2,1960,final,at,mc,at,mc,0,NaN,NaN,1960,...,4.111648e+06,2.295892e+06,MCO,..,21808,852.872041,0.111111,0.200000,0.333333,0.200000
3,1960,final,at,no,at,no,1,NaN,NaN,1960,...,4.362948e+06,4.091117e+06,NOR,1451.2850779894,3581239,1353.489212,0.292683,0.200000,0.333333,0.452830
4,1960,final,at,de,at,de,2,NaN,NaN,1960,...,4.551651e+06,3.273670e+06,DEU,1162.12266831735,72814900,524.812724,0.666667,0.666667,0.800000,0.800000


In [59]:
vote_dropped = vote_dists2.drop(["from_country_id", "to_country_id", "Country Code_to", "Country Code_from", "iso-alpha-2_from", "iso-alpha-2_to", "name_to", "name_from", "capital_to", "capital_from", "geometry_from", "geometry_to"], axis=1)
vote_dropped

,year,round,from_country,to_country,total_points,tele_points,jury_points,Year_from,iso-alpha-3_from,chrstprotpct_from,...,city_to,lat_to,lon_to,GDP per capita (current US$)_to,"Population, total_to",distance,jaccard_artists,jaccard_genre,dice_genre,dice_artists
0,1960,final,at,fr,1,NaN,NaN,1960,AUT,0.0620,...,Paris,3.760847e+06,2.889644e+06,1302.53149729892,47412964,1036.040752,0.153846,0.090909,0.166667,0.266667
1,1960,final,at,gb,3,NaN,NaN,1960,AUT,0.0620,...,London,3.620981e+06,3.203213e+06,1397.5948032844,52400000,1237.492544,0.302326,0.224490,0.366667,0.464286
2,1960,final,at,mc,0,NaN,NaN,1960,AUT,0.0620,...,Monaco,4.111648e+06,2.295892e+06,..,21808,852.872041,0.111111,0.200000,0.333333,0.200000
3,1960,final,at,no,1,NaN,NaN,1960,AUT,0.0620,...,Oslo,4.362948e+06,4.091117e+06,1451.2850779894,3581239,1353.489212,0.292683,0.200000,0.333333,0.452830
4,1960,final,at,de,2,NaN,NaN,1960,AUT,0.0620,...,Berlin,4.551651e+06,3.273670e+06,1162.12266831735,72814900,524.812724,0.666667,0.666667,0.800000,0.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47416,2023,final,gb,si,1,0.0,1.0,2023,GBR,0.0727,...,Ljubljana,4.670582e+06,2.559758e+06,32610.110922729,2120461,1231.136216,0.200000,0.250000,0.400000,0.333333
47417,2023,final,gb,al,0,0.0,0.0,2023,GBR,0.0727,...,Tirana,5.143798e+06,2.078711e+06,8575.17113446213,2745972,1893.007274,0.191489,0.363636,0.533333,0.321429
47418,2023,final,gb,pt,0,0.0,0.0,2023,GBR,0.0727,...,Lisbon,2.664944e+06,1.947184e+06,27331.2094945255,10578174,1578.485825,0.098039,0.111111,0.200000,0.178571
47419,2023,final,gb,rs,0,0.0,0.0,2023,GBR,0.0727,...,Belgrade,5.147211e+06,2.470142e+06,12281.5058488303,6623183,1693.153915,0.200000,0.071429,0.133333,0.333333


In [60]:

vote_dropped.to_csv("../data/fulldata.csv")